Step 1: Importing dependencies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
from scipy import stats
from google.colab import drive
import os

Step 2: Visualization config

In [ ]:
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

Step 3: defining variable and preparing csv file

In [ ]:
csv_path = "/content/synthetic_stock_data.csv"

Step 4: first visualization data

In [ ]:
# --- Verificar si el archivo existe en la ruta ---
if os.path.exists(csv_path):
    print(f"Archivo encontrado en: {csv_path}")
    # --- Cargar el conjunto de datos ---
    try:
        df = pd.read_csv(csv_path)
        print("¡DataFrame cargado exitosamente!")

        # ==================================================
        # 1.    Carga inicial del proyecto.
        # ==================================================
        print("\n----- 1. Carga e Inspección Inicial -----")

        # Ver las primeras y últimas filas
        print("\nPrimeras 5 filas:")
        print(df.head())
        print("\nÚltimas 5 filas:")
        print(df.tail())

        # Obtener dimensiones
        print(f"\nDimensiones del DataFrame (filas, columnas): {df.shape}")

        # Revisar tipos de datos y valores no nulos
        print("\nInformación del DataFrame (Tipos de datos y Nulos):")
        df.info()

        # Obtener estadísticas descriptivas básicas para variables numéricas
        print("\nEstadísticas Descriptivas (Variables Numéricas):")
        print(df.describe().T) # Usar .T para transponer y facilitar la lectura

        # Contar valores únicos por columna
        print("\nConteo de Valores Únicos por Columna:")
        print(df.nunique())

        # ... (Aquí pegarías el resto del código del EDA: Limpieza, Univariado, Bivariado, etc.) ...
        # El resto del código que te proporcioné anteriormente funcionará
        # a partir de aquí, ya que la variable 'df' ahora contiene tus datos.

    except Exception as e:
        print(f"Error al leer el archivo CSV: {e}")
        print("Asegúrate de que el archivo se subió correctamente y no está corrupto.")

else:
    print(f"¡ERROR! No se encontró el archivo en la ruta: {csv_path}")
    print("Por favor, asegúrate de haber subido el archivo 'synthetic_stock_data.csv' a tu sesión actual de Colab.")
    print("Puedes hacerlo usando el botón 'Subir archivo' en el panel de archivos de la izquierda.")

Step 5: handling duplicates

In [ ]:
# --- Manejo de Duplicados ---
print("\nNúmero de Filas Duplicadas:")
duplicates = df.duplicated().sum()
print(duplicates)

if duplicates > 0:
    print(f"\nSe encontraron {duplicates} filas duplicadas. Eliminándolas.")
    df.drop_duplicates(inplace=True)
    print(f"Nuevas dimensiones del DataFrame: {df.shape}")
else:
    print("\nNo se encontraron filas duplicadas.")

# --- Corrección de Tipos de Datos ---
print("\nVerificando y corrigiendo tipos de datos...")
# Convertir 'Date' a datetime si no lo está ya
if not pd.api.types.is_datetime64_any_dtype(df['Date']):
    try:
        df['Date'] = pd.to_datetime(df['Date'])
        print("'Date' convertida a tipo datetime.")
    except Exception as e:
        print(f"Error al convertir 'Date': {e}. Se mantendrá como objeto.")
else:
    print("'Date' ya es de tipo datetime.")

Step 6: Univariate analysis

In [ ]:
# ==================================================
# 2. Análisis Univariado
# ==================================================
print("\n----- 2. Análisis Univariado -----")

# Identificar columnas numéricas y categóricas (RE-IDENTIFICAR después de limpieza/corrección)
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes(include='object').columns.tolist()
date_col = 'Date' # Definir explícitamente
# Asegurarse de no incluir columnas no deseadas si su tipo cambió
if 'Year' in numeric_cols: numeric_cols.remove('Year') # Si se crearon en una ejecución previa
if 'Month' in numeric_cols: numeric_cols.remove('Month')
if 'Day' in numeric_cols: numeric_cols.remove('Day')
if 'DayOfWeek' in numeric_cols: numeric_cols.remove('DayOfWeek')

print(f"\nColumnas Numéricas para Análisis Univariado: {numeric_cols}")
print(f"Columnas Categóricas para Análisis Univariado: {categorical_cols}")

# --- Variables Numéricas ---
print("\nAnálisis de Variables Numéricas:")

for col in numeric_cols:
    print(f"\n--- Análisis de '{col}' ---")
    if df[col].isnull().any():
        print(f"Advertencia: La columna '{col}' contiene valores NaN. Las estadísticas pueden verse afectadas.")
        # Considera df[col].dropna().skew() o .kurt() si quieres estadísticas sin NaNs
        skewness = "N/A (contiene NaN)"
        kurtosis = "N/A (contiene NaN)"
    else:
        skewness = f"{df[col].skew():.2f}"
        kurtosis = f"{df[col].kurt():.2f}"

    print(f"Media: {df[col].mean():.2f}")
    print(f"Mediana: {df[col].median():.2f}")
    print(f"Desviación Estándar: {df[col].std():.2f}")
    print(f"Mínimo: {df[col].min():.2f}")
    print(f"Máximo: {df[col].max():.2f}")
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    print(f"Q1: {Q1:.2f}")
    print(f"Q3: {Q3:.2f}")
    print(f"IQR (Rango Intercuartílico): {IQR:.2f}")
    print(f"Asimetría (Skewness): {skewness}")
    print(f"Curtosis: {kurtosis}")

    # Visualización: Histograma y Boxplot
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    # Usar dropna() para evitar errores en el plot si hay NaNs
    sns.histplot(df[col].dropna(), kde=True)
    plt.title(f'Distribución de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')

    plt.subplot(1, 2, 2)
    sns.boxplot(y=df[col].dropna()) # Usar dropna() aquí también
    plt.title(f'Boxplot de {col}')
    plt.ylabel(col)

    plt.tight_layout()
    plt.show()

# --- Variables Categóricas ---
print("\nAnálisis de Variables Categóricas:")

for col in categorical_cols:
    print(f"\n--- Análisis de '{col}' ---")

    # Frecuencias y Proporciones
    print("Conteo de Valores:")
    value_counts = df[col].value_counts()
    print(value_counts)
    print("\nProporción de Valores (%):")
    print(value_counts.div(value_counts.sum()).mul(100).round(2).astype(str) + '%')

    # Visualización: Gráfico de Barras (si no hay demasiadas categorías)
    num_unique = df[col].nunique()
    print(f"Número de categorías únicas: {num_unique}")
    if num_unique > 0 and num_unique < 30: # Umbral arbitrario y evitar error si columna está vacía
        plt.figure(figsize=(10, max(5, num_unique * 0.3))) # Ajustar altura dinámicamente
        sns.countplot(y=df[col], order = value_counts.index, palette='viridis')
        plt.title(f'Frecuencia de Categorías en {col}')
        plt.xlabel('Conteo')
        plt.ylabel(col)
        plt.tight_layout()
        plt.show()
    elif num_unique >= 30:
        print(f"No se genera gráfico de barras para '{col}' debido a la alta cardinalidad ({num_unique} categorías).")
        print("Top 10 categorías:")
        print(value_counts.head(10))
    else:
         print(f"La columna '{col}' no tiene valores o categorías únicas para graficar.")

Step 7: Bivariate analysis

In [ ]:
# ==================================================
# 3. Análisis Bivariado
# ==================================================
print("\n----- 3. Análisis Bivariado -----")

# --- Numérica vs. Numérica ---
print("\nAnálisis Numérica vs. Numérica:")

# Matriz de correlación y Mapa de calor
print("\nMatriz de Correlación:")
# Seleccionar solo columnas numéricas que existen en el df actual
existing_numeric_cols = [col for col in numeric_cols if col in df.columns]
if existing_numeric_cols:
    correlation_matrix = df[existing_numeric_cols].corr()
    print(correlation_matrix)

    print("\nMapa de Calor de Correlación:")
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, annot_kws={"size": 8})
    plt.title('Mapa de Calor de Correlación entre Variables Numéricas')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
else:
    print("No se encontraron columnas numéricas para calcular la correlación.")


# Diagramas de dispersión (Seleccionar pares relevantes)
print("\nDiagramas de Dispersión (Ejemplos):")
# Verificar si las columnas existen antes de intentar plotear
scatter_pairs = [('Open', 'Close'), ('Volume', 'Volatility'), ('PE_Ratio', 'Dividend_Yield')]
plt.figure(figsize=(18, 6))
plot_index = 1
for x_col, y_col in scatter_pairs:
    if x_col in df.columns and y_col in df.columns:
        plt.subplot(1, len(scatter_pairs), plot_index)
        sns.scatterplot(data=df, x=x_col, y=y_col, alpha=0.5)
        plt.title(f'{x_col} vs {y_col}')
        plot_index += 1
    else:
        print(f"Advertencia: No se pueden plotear '{x_col}' vs '{y_col}' (una o ambas columnas faltan).")
if plot_index > 1: # Solo mostrar si se ploteó algo
    plt.tight_layout()
    plt.show()
else:
    print("No se pudieron generar diagramas de dispersión de ejemplo.")


# --- Categórica vs. Categórica ---
print("\nAnálisis Categórica vs. Categórica:")

# Ejemplo: Sector vs Trend
if 'Sector' in categorical_cols and 'Trend' in categorical_cols:
    print("\nTabla de Contingencia: Sector vs Trend")
    try:
        crosstab_st = pd.crosstab(df['Sector'], df['Trend'])
        print(crosstab_st)

        # Gráfico de barras agrupadas
        crosstab_st.plot(kind='bar', figsize=(12, 7), rot=45)
        plt.title('Distribución de Trend por Sector')
        plt.xlabel('Sector')
        plt.ylabel('Conteo')
        plt.legend(title='Trend')
        plt.tight_layout()
        plt.show()

        # Prueba Chi-cuadrado (opcional)
        try:
            chi2, p, dof, expected = stats.chi2_contingency(crosstab_st)
            print(f"\nPrueba Chi-cuadrado (Sector vs Trend): Chi2={chi2:.2f}, p-value={p:.3f}")
            if p < 0.05:
                print("Hay evidencia de una asociación significativa entre Sector y Trend (p < 0.05).")
            else:
                print("No hay evidencia suficiente de una asociación significativa entre Sector y Trend (p >= 0.05).")
        except ValueError as ve:
             print(f"No se pudo realizar la prueba Chi-cuadrado (posiblemente por ceros en tabla): {ve}")
        except Exception as e:
             print(f"Error inesperado en la prueba Chi-cuadrado: {e}")

    except Exception as e:
        print(f"Error al generar la tabla de contingencia Sector vs Trend: {e}")
else:
    print("No se encontraron las columnas 'Sector' y/o 'Trend' para el análisis Categórico-Categórico.")


# --- Numérica vs. Categórica ---
print("\nAnálisis Numérica vs. Categórica:")

# Ejemplo: Close vs Sector
if 'Sector' in categorical_cols and 'Close' in existing_numeric_cols:
    plt.figure(figsize=(12, 6))
    # Ordenar por mediana para mejor visualización
    order = df.groupby('Sector')['Close'].median().sort_values().index
    sns.boxplot(data=df, x='Sector', y='Close', order=order, palette='viridis')
    plt.title('Distribución de Close por Sector (Ordenado por Mediana)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print("\nEstadísticas Descriptivas de 'Close' Agrupadas por 'Sector':")
    print(df.groupby('Sector')['Close'].describe())
else:
     print("No se encontraron las columnas 'Sector' y/o 'Close' para el análisis Numérico-Categórico.")


# Ejemplo: Sentiment_Score vs Trend
if 'Trend' in categorical_cols and 'Sentiment_Score' in existing_numeric_cols:
    plt.figure(figsize=(10, 6))
    sns.violinplot(data=df, x='Trend', y='Sentiment_Score', palette='muted', inner='quartile') # Mostrar cuartiles
    plt.title('Distribución de Sentiment_Score por Trend')
    plt.tight_layout()
    plt.show()

    print("\nEstadísticas Descriptivas de 'Sentiment_Score' Agrupadas por 'Trend':")
    print(df.groupby('Trend')['Sentiment_Score'].describe())
else:
     print("No se encontraron las columnas 'Trend' y/o 'Sentiment_Score' para el análisis Numérico-Categórico.")

Step 8: Outliers handler

In [ ]:
# ==================================================
# 4. Identificación y Manejo de Outliers (Más a fondo)
# ==================================================
print("\n----- 4. Identificación y Manejo de Outliers -----")

print("\nIdentificación de Outliers usando el método IQR:")
# Revisaremos algunas columnas clave como ejemplo
outlier_check_cols = ['Close', 'Volume', 'PE_Ratio', 'Volatility', 'Sentiment_Score', 'Market_Cap', 'Dividend_Yield']
outlier_summary = {}

for col in outlier_check_cols:
    if col in df.columns and pd.api.types.is_numeric_dtype(df[col]) and not df[col].isnull().all():
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Evitar errores si IQR es 0
        if IQR > 0:
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
            outlier_count = outliers.shape[0]
            percentage = (outlier_count / df[col].notna().sum()) * 100 if df[col].notna().sum() > 0 else 0
            outlier_summary[col] = {'count': outlier_count, 'percentage': percentage}

            print(f"\nOutliers en '{col}': {outlier_count} ({percentage:.2f}%)")
            print(f"  Límites (1.5*IQR): Inferior={lower_bound:.2f}, Superior={upper_bound:.2f}")
            if outlier_count > 0 and outlier_count < 15: # Mostrar algunos si no son demasiados
                 print("  Algunos ejemplos de outliers:")
                 # Intentar mostrar Date si existe y es datetime
                 cols_to_show = [col for col in [date_col,'Company', col] if col in outliers.columns]
                 print(outliers[cols_to_show].head())
            elif outlier_count >= 15:
                 print(f"  Se encontraron {outlier_count} outliers, no se muestran ejemplos por brevedad.")
                 print(f"  Valor Mínimo Outlier (si existe < lower): {outliers[outliers[col] < lower_bound][col].min() if any(outliers[col] < lower_bound) else 'N/A'}")
                 print(f"  Valor Máximo Outlier (si existe > upper): {outliers[outliers[col] > upper_bound][col].max() if any(outliers[col] > upper_bound) else 'N/A'}")

        else:
            print(f"\n'{col}': IQR es 0, no se pueden calcular límites basados en IQR.")
            outlier_summary[col] = {'count': 0, 'percentage': 0.0}
    elif col not in df.columns:
         print(f"\nAdvertencia: La columna '{col}' no existe en el DataFrame.")
    elif not pd.api.types.is_numeric_dtype(df[col]):
         print(f"\nAdvertencia: La columna '{col}' no es numérica.")
    else: # Es numérica pero todos son NaN
         print(f"\nAdvertencia: La columna '{col}' solo contiene valores NaN.")


print("\nResumen de Outliers (método 1.5*IQR):")
for col, data in outlier_summary.items():
    print(f"- {col}: {data['count']} outliers ({data['percentage']:.2f}%)")


print("\nNota sobre el manejo de outliers:")
print("El tratamiento de outliers (eliminación, transformación, capping, uso de modelos robustos)")
print("depende críticamente del contexto del análisis, del dominio del problema y del objetivo final (ej: modelado).")
print("En este EDA inicial, solo los identificamos. Los boxplots del análisis univariado también ayudan a visualizarlos.")